In [ ]:
import asyncio
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
file_paths = [
    "data/Eat-That-Frog.pdf",
    "data/Master_your_Time.pdf",
    "data/The-7-habits-of-highly-effective-people.pdf",
    "data/Atomic_habits"
]

async def load_all_pdfs(file_paths):
    pages = []
    for file_path in file_paths:
        loader = PyPDFLoader(file_path)
        async for page in loader.alazy_load():
            pages.append(page)
    return pages




In [ ]:
# Run the async function
pages = await load_all_pdfs(file_paths)

In [ ]:
pages

# In general, asynchronous -- from Greek asyn- ("not with/together") and chronos ("time") -- describes objects or events not coordinated in time.

In [ ]:

print(pages[5].page_content)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(pages)
print(f'Total numberof chunks: {len(docs)}')

In [ ]:
print('The ability to \nconcentrate single-mindedly on your most important task, to do it \nwell and to finish it completely, is the key to great success, \nachievement, respect, status and happiness in life')

In [ ]:
print("Guaranteed. \nThere will be no limit to what you can accomplish when you learn \nhow to “Eat That Frog!” \t (Brian Tracy)")

In [ ]:
len(pages), len(docs)

# Load the Generated Embeddings

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

In [ ]:
# Create the same embeddings object as used in the FAISS index
embedding = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# Load the FAISS index from the file
vectorstore = FAISS.load_local('my_faiss_index', embedding, allow_dangerous_deserialization=True)

In [ ]:
# Example of using the vectorstore to get similar documnts
docs = vectorstore.similarity_search("How to create a time management system?", k=3)

for i, doc in enumerate(docs, 1):
    print(f"Chunk {i}:")
    print([doc.page_content])
    print("\n---\n")



In [ ]:
# Example of using the vectorstore to search for similar documents
docs = vectorstore.similarity_search("What is the Secret of Success", k=3)


for i, doc in enumerate(docs, 1):
    print(f'Document {i}:')
    if doc:
        print([doc.page_content])

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following context to answer the question."
    "If you don't know the answer, say 'I don't know'."
    "If the question is not related to the context, say 'I can't help you with that.'"
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
# fetch docs from the retriever
def get_context_from_retriever(query):
    docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context

In [ ]:
get_context_from_retriever("who wrote Eat that Frog?")

In [ ]:
# Use that context with your prompt
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following context to answer the question."
    "If you don't know the answer, say 'I don't know'."
    "If the question is not related to the context, say 'I can't help you with that.'"
    "\n\n"
    "Context: {context}"
)


prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [ ]:
import openai
from dotenv import load_dotenv
load_dotenv()
import os
def get_llm_response(prompt):
    client = openai.OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
    )
    response = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[
            {"role": "system", "content": "You are a helpful movie assistant, which provides movie recommendations and insights in a brief and concise manner."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

In [ ]:
from dotenv import load_dotenv
load_dotenv()
print(os.getenv("OPENROUTER_API_KEY"))


In [ ]:
# Combine Everything
def final_response(user_query):
    context = get_context_from_retriever(user_query)
    formatted_prompt = prompt_template.format_messages(input=user_query, context=context)

    # join message content for your LLM API
    full_prompt = "\n".join([m.content for m in formatted_prompt])

    return get_llm_response(full_prompt)

In [ ]:
response = final_response("What does mean by Eat that frog?")
print(response)

In [ ]:
client = openai.OpenAI(
    api_key=os.getenv('OPENROUTER_API_KEY'), 
    base_url="https://openrouter.ai/api/v1"
)

In [ ]:
# Continuous chat loop
while True:
    print("\nYou: ", end="", flush=True)  # Ensure input prompt appears
    user_input = input()  # Take user input

    if user_input.lower() in ["exit", "quit"]:  # Exit condition
        print("Exiting chat. Goodbye!")
        break

    # Send request to OpenRouter API
    response = client.chat.completions.create(
        model="google/gemini-2.5-pro-exp-03-25:free",  # Your chosen model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_input}
        ]
    )

    # Print AI response
    print("\nAI:", response.choices[0].message.content, flush=True)